In [ ]:
!pip install --quiet streamlit torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from google.colab import drive
import os

@st.cache_resource
def load_model_and_tokenizer():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Paths to your model and tokenizer in Google Drive
    base_path = '/content/drive/MyDrive/Question Generator'
    model_path = os.path.join(base_path, 't5_model_base')
    tokenizer_path = os.path.join(base_path, 't5_tokenizer_base')

    if not os.path.exists(model_path) or not os.path.exists(tokenizer_path):
        raise FileNotFoundError(f"Model or tokenizer not found in {base_path}. Please check the path.")

    model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)
    tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
    return model, tokenizer, device

def get_question(context, answer, model, tokenizer, device):
    text = f"context: {context} answer: {answer}"
    encoding = tokenizer.encode_plus(text, max_length=512, padding='max_length',
                                     truncation=True, return_tensors="pt").to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    outs = model.generate(input_ids=input_ids,
                          attention_mask=attention_mask,
                          max_length=72,
                          num_beams=5,
                          early_stopping=True)
    question = tokenizer.decode(outs[0], skip_special_tokens=True).replace("question:", "").strip()
    return question

# Streamlit UI
st.title("Question Generator")

# Load model and tokenizer
try:
    model, tokenizer, device = load_model_and_tokenizer()
    st.success("Model and tokenizer loaded successfully!")
except Exception as e:
    st.error(f"Error loading model and tokenizer: {str(e)}")
    st.stop()

context = st.text_area("Enter the context:", "Donald Trump is an American media personality and businessman who served as the 45th president of the United States.")
answer = st.text_input("Enter the answer:", "Donald Trump")

if st.button("Generate Question"):
    with st.spinner("Generating question..."):
        question = get_question(context, answer, model, tokenizer, device)
    st.success("Question generated!")
    st.write("Generated Question:", question)

Overwriting app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

130.211.248.45


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://130.211.248.45:8501

npx: installed 22 in 2.312s
your url is: https://busy-paws-shake.loca.lt
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
